In [1]:
import sys
sys.path.append('..')

In [11]:
import os
import numpy as np

import torch
from torch.utils.data import DataLoader

from tqdm import tqdm

from SMPL.smpl_torch_batch import SMPLModel

from obj_utils.smpl import *
from obj_utils.projection import *

from generation import SMPLDataset

In [3]:
# check cuda
print(torch.cuda.is_available())

True


### Load Betas

In [4]:
dataset_path = './dataset.npz'
dataset = np.load(dataset_path)

betas = dataset['betas']
print(betas.shape)

(25135, 10)


### Make Verts

In [5]:
smpl_model_path = '../SMPL/model.pkl'
smpl_model = SMPLModel(device='cuda', model_path=smpl_model_path)

In [9]:
def smpl_reconstruction(betas, poses, smpl_model: SMPLModel, batch_size = 512):
    smpl_dataset = SMPLDataset(beta_arr=betas, pose_arr=poses, device=smpl_model.device)
    smpl_dataloader = DataLoader(smpl_dataset, batch_size=batch_size, shuffle=False)

    smpl_meshes = []
    pbar = tqdm(smpl_dataloader)
    for parameters_batch in pbar:
        batch_vertices, _ = smpl_parameters_batch_to_vertices_and_joints(parameters_batch, smpl_model)
        
        for vertices in batch_vertices:
            smpl_meshes.append(vertices)
    pbar.close()

    smpl_meshes = np.array(smpl_meshes)
    return smpl_meshes

In [10]:
poses = get_A_pose(betas.shape[0])
verts = smpl_reconstruction(betas, poses, smpl_model)

print(verts.shape)

100%|██████████| 50/50 [00:02<00:00, 22.84it/s]


(25135, 6890, 3)


### Make Silhouettes

In [14]:
faces = smpl_model.faces

for i, v in enumerate(tqdm(verts)):
    v -= (v.min(axis=0) + v.max(axis=0)) / 2
    h = v[:, 1].max() - v[:, 1].min()

    v_f = v.copy()
    v_l = v.copy()[:, ::-1]

    v_f[:, 2] += 3.
    v_f[:, 2] = 6. - v_f[:, 2]
    v_l[:, 2] += 3.

    p_f = pinhole(v_f)
    p_l = pinhole(v_l)

    s_f = make_silhouette(p_f, faces, 500, 600, h, 2.3)
    s_l = make_silhouette(p_l, faces, 500, 600, h, 2.3)

    save_path = os.path.join('silhouettes', f'{i:05d}')
    os.makedirs(save_path, exist_ok=True)

    s_f.save(os.path.join(save_path, 'frontal.png'))
    s_l.save(os.path.join(save_path, 'lateral.png'))

100%|██████████| 25135/25135 [28:38<00:00, 14.63it/s]
